In [6]:
from sentence_transformers import SentenceTransformer, util
from transformers import BertTokenizer
import pandas as pd
import numpy as np
import nltk
import math
from joblib import Parallel, delayed

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')

from nltk.tokenize import word_tokenize
from string import punctuation
import nltk
from unicodedata import normalize
from nltk.stem import RSLPStemmer
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer

import torch
import time


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [7]:
df_assunto= pd.read_table("dados-conle-anonimizado-assunto-notnull - dados-conle-anonimizado-assunto-notnull.tsv",sep=";")

In [8]:
base = pd.read_csv("base_20230428_douglas.csv", sep=',')

In [118]:
base

,codProposicao,txtSiglaTipo,numAno,numNumero,txtNome,txtEmenta,txtExplicacaoEmenta,txtIndexacao,txtInteiroTeor
0,16357,PL,1999,1165,PL 1165/1999,"Altera dispositivo da Lei nº 8.987, de 13 de f...",Estabelece que as concessionárias disponibiliz...,"Alteração, Lei das Concessões de Serviços Públ...",Ofício nº 1416 (SF) ...
1,19098,PL,1992,3097,PL 3097/1992,Dispõe sobre a eleição de diretores de fundos ...,NaN,"NORMAS, ELEIÇÃO DIRETA, EMPREGADO, APOSENTADO,...",COMISSÃO DE CONSTITUIÇÃO E JUSTIÇA E DE REDAÇÃ...
2,20464,PL,2000,3927,PL 3927/2000,Altera a composição dos Tribunais Regionais do...,"Altera a composição do TRT da 5ª região, 6ª re...","Alteração, Lei Federal, composição, Tribunal R...","COMISSÃO DE TRABALHO, DE ADMINISTRAÇÃO E SERVI..."
3,20683,PL,1998,4117,PL 4117/1998,Dispõe sobre o acesso a ambientes de uso colet...,NaN,"Autorização, pessoa portadora de deficiência, ...",Câmara dos Deputados\n Departamento ...
4,20857,PL,1998,4395,PL 4395/1998,Estabelece as Diretrizes Nacionais de Defesa C...,NaN,"Fixação, diretrizes, defesa civil, (Sindec), d...",- 1 - \nCOMISSÃO DE CONSTITUIÇÃO E JUSTIÇA E D...
...,...,...,...,...,...,...,...,...,...
57104,2358873,PL,2023,2233,PL 2233/2023,Altera o parágrafo único do art. 71 do Código ...,NaN,NaN,1 \n \n \n \n \nCÂMARA DOS DEPUTADOS \nPROJET...
57105,2358874,PL,2019,3616,PL 3616/2019,"Altera a Lei nº 9.503, de 23 de setembro de 19...",NaN,NaN,"Altera a Lei nº 9.503, de 23 de setembro\nde 1..."
57106,2358875,PL,2019,1822,PL 1822/2019,"Altera a Lei nº 11.340, de 7 de agosto de 2006...",NaN,NaN,"Altera a Lei nº 11.340, de 7 de agosto de\n200..."
57107,2358877,PL,2019,3815,PL 3815/2019,"Altera a Lei nº 7.565, de 19 de dezembro de 19...",NaN,NaN,"Altera a Lei nº 7.565, de 19 de dezembro\nde ..."


In [119]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [120]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [121]:
arr_assunto = df_assunto.to_numpy()
y,X = arr_assunto[:,0],arr_assunto[:,1]
y = [i.strip() for i in y]

In [122]:
device_gpu = torch.device("cuda")

## Search

In [124]:
def search(query, data_emb,base,top_k=20):
    #hits = util.semantic_search(query_emb,data,top_k=k)
    query_emb = model.encode(query,convert_to_tensor=True,device=device_gpu)
    hits = util.cos_sim(query_emb,data_emb)[0]
    rank = torch.topk(hits, k=top_k)
    
    
    top_n = list()
    for i in range(top_k):
        dado = dict()
        dado['txtNome'] = base.iloc[rank.indices[i].item(),4]
        dado['score'] = rank.values[i].item()
        top_n.append(dado)
    return top_n
    

## Recall

In [125]:
def verificar(y,top_n):
    for d in top_n:
        if y == d["txtNome"]:
            return 1
    return 0

In [126]:
def avaliacaoRecall(isPre,top_k=20):

    quant_encontrado=0
    quant_relevante =0
    a = time.time()
    for l,x in zip(y,X):
    
        query_tokenized = x
        if isPre:
            query_tokenized = preprocess(x)
            
        top_n = search(query_tokenized,data_emb,base,top_k)
                
        quant_relevante+=1
        quant_encontrado+=verificar(l,top_n)
    d = time.time()
    print("Duração: %f" %(d-a))
    recall = quant_encontrado / quant_relevante
    print("Recall: "+str(recall))

In [15]:
def avaliacaoRR(isPre):

    quant_encontrado=0
    for l,x in zip(y,X):
    
        query_tokenized = x
        if isPre:
            query_tokenized = preprocess(x)
            
        top_n = search(query_tokenized,data_emb,base)
        if top_n[0]["txtNome"].strip() == l.strip():
            quant_encontrado+=1
    rr = quant_encontrado / 295
    print("RR: %f" % (rr))

In [16]:
def avaliacaoPrecision(isPre,top_k=20):

    soma=0
    for l,x in zip(y,X):
    
        query_tokenized = x
        if isPre:
            query_tokenized = preprocess(x)
            
        top_n = search(query_tokenized,data_emb,base,top_k)
        encontrado=verificar(l,top_n)
        soma+=(encontrado/top_k)
    precision = soma / 295
    print("Precision: "+str(precision))

In [17]:
def avaliacaoMAP(isPre,top_k=20):
    l_v = list()
    labels_nsL = list()
    
    for l,x in zip(y,X):
    
        query_tokenized = x
        if isPre:
            query_tokenized = preprocess(x)
            
        top_n = search(query_tokenized,data_emb,base,top_k)
        l_v.append(l)
        labels_nsL.append([doc["txtNome"] for doc in top_n])
        
    denominador = 0
    encontrou=False
    soma=0
    for Y,x in zip(l_v,labels_nsL):
        calc = 0
        quant=0
        for k in x:
            if str(k).strip() == Y.strip():
                quant=1
                encontrou=True
            denominador+=1
            calc=quant/denominador
            if encontrou:
                calc = calc * 1
            else:
                calc = calc * 0
            soma+=calc
            encontrou = False
    MAP = soma /295
    print("MAP@ %d: %f" % (top_k,MAP))

In [277]:
data = base.txtInteiroTeor.fillna("Em branco")        # Trocar com nome da coluna desejada

## Config 1

In [278]:
a = time.time()

In [279]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [280]:
d = time.time()

In [281]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 1267.635002


## Avaliação

### Recall

In [282]:
avaliacaoRecall(False,top_k=5)

Duração: 25.875997
Recall: 0.2576271186440678


In [283]:
avaliacaoRecall(False,top_k=10)

Duração: 25.996002
Recall: 0.30847457627118646


In [284]:
avaliacaoRecall(False,top_k=20)

Duração: 29.375000
Recall: 0.3593220338983051


### MAP

In [285]:
avaliacaoMAP(False,top_k=5)

MAP@ 5: 0.000887


In [286]:
avaliacaoMAP(False,top_k=10)

MAP@ 10: 0.000516


In [287]:
avaliacaoMAP(False,top_k=20)

MAP@ 20: 0.000288


### Precision

In [288]:
avaliacaoPrecision(False,top_k=5)

Precision: 0.05152542372881348


In [289]:
avaliacaoPrecision(False,top_k=10)

Precision: 0.030847457627118588


In [290]:
avaliacaoPrecision(False,top_k=20)

Precision: 0.017966101694915217


### RR

In [291]:
avaliacaoRR(False)

RR: 0.145763


## Config 5

In [292]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]

    return " ".join(terms)

In [293]:
data_preprocess = Parallel(n_jobs=4)(delayed(preprocess)(doc) for doc in data)

In [294]:
a = time.time()

In [295]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [296]:
d = time.time()

In [297]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 1265.157001


## Avaliação

### Recall

In [298]:
avaliacaoRecall(True,top_k=5)

Duração: 25.439997
Recall: 0.2135593220338983


In [299]:
avaliacaoRecall(True,top_k=10)

Duração: 25.954997
Recall: 0.2576271186440678


In [300]:
avaliacaoRecall(True,top_k=20)

Duração: 28.873000
Recall: 0.29491525423728815


### MAP

In [301]:
avaliacaoMAP(True,top_k=5)

MAP@ 5: 0.000544


In [302]:
avaliacaoMAP(True,top_k=10)

MAP@ 10: 0.000415


In [303]:
avaliacaoMAP(True,top_k=20)

MAP@ 20: 0.000279


### Precision

In [304]:
avaliacaoPrecision(True,top_k=5)

Precision: 0.04271186440677962


In [305]:
avaliacaoPrecision(True,top_k=10)

Precision: 0.02576271186440674


In [306]:
avaliacaoPrecision(True,top_k=20)

Precision: 0.01474576271186438


### RR

In [307]:
avaliacaoRR(True)

RR: 0.125424


## Config 0

In [308]:
def preprocess(txt):
    txt = str(txt)
    
    stopwords = nltk.corpus.stopwords.words("portuguese")
   
    terms = word_tokenize(txt)
    terms = [word for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [309]:
data_preprocess = Parallel(n_jobs=4)(delayed(preprocess)(doc) for doc in data)

In [310]:
a = time.time()

In [311]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [312]:
d = time.time()

In [313]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 882.433486


## Avaliação

### Recall

In [314]:
avaliacaoRecall(True,top_k=5)

Duração: 15.234998
Recall: 0.2542372881355932


In [315]:
avaliacaoRecall(True,top_k=10)

Duração: 15.474998
Recall: 0.31186440677966104


In [316]:
avaliacaoRecall(True,top_k=20)

Duração: 16.570065
Recall: 0.34576271186440677


### MAP

In [317]:
avaliacaoMAP(True,top_k=5)

MAP@ 5: 0.000700


In [318]:
avaliacaoMAP(True,top_k=10)

MAP@ 10: 0.000515


In [319]:
avaliacaoMAP(True,top_k=20)

MAP@ 20: 0.000280


### Precision

In [320]:
avaliacaoPrecision(True,top_k=5)

Precision: 0.05084745762711857


In [321]:
avaliacaoPrecision(True,top_k=10)

Precision: 0.031186440677966044


In [322]:
avaliacaoPrecision(True,top_k=20)

Precision: 0.017288135593220306


### RR

In [323]:
avaliacaoRR(True)

RR: 0.135593


# TxtEmenta

In [130]:
data = base.txtEmenta.fillna("Em branco")

## Config 1

In [131]:
a = time.time()

In [132]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [133]:
d = time.time()

In [134]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 276.334136


## Avaliação

### Recall

In [135]:
avaliacaoRecall(False,top_k=5)

Duração: 61.823027
Recall: 0.20677966101694914


In [136]:
avaliacaoRecall(False,top_k=10)

Duração: 97.622998
Recall: 0.2711864406779661


In [137]:
avaliacaoRecall(False,top_k=20)

Duração: 98.610999
Recall: 0.31186440677966104


### MAP

In [138]:
avaliacaoMAP(False,top_k=5)

MAP@ 5: 0.000506


In [139]:
avaliacaoMAP(False,top_k=10)

MAP@ 10: 0.000324


In [140]:
avaliacaoMAP(False,top_k=20)

MAP@ 20: 0.000237


### Precision

In [141]:
avaliacaoPrecision(False,top_k=5)

Precision: 0.04135593220338979


In [142]:
avaliacaoPrecision(False,top_k=10)

Precision: 0.027118644067796568


In [143]:
avaliacaoPrecision(False,top_k=20)

Precision: 0.015593220338983022


### RR

In [144]:
avaliacaoRR(False)

RR: 0.118644


In [145]:
avaliacaoRR(False)

RR: 0.118644


## Config 5

In [146]:
data = base.txtEmenta.fillna("Em branco")

In [147]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]

    return " ".join(terms)

In [148]:
data_preprocess = Parallel(n_jobs=4)(delayed(preprocess)(doc) for doc in data)

In [149]:
a = time.time()

In [150]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [151]:
d = time.time()

In [152]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 174.720998


## Avaliação

### Recall

In [153]:
avaliacaoRecall(True,top_k=5)

Duração: 13.701998
Recall: 0.1864406779661017


In [154]:
avaliacaoRecall(True,top_k=10)

Duração: 15.682001
Recall: 0.23389830508474577


In [155]:
avaliacaoRecall(True,top_k=20)

Duração: 11.026998
Recall: 0.288135593220339


### MAP

In [156]:
avaliacaoMAP(True,top_k=5)

MAP@ 5: 0.000493


In [157]:
avaliacaoMAP(True,top_k=10)

MAP@ 10: 0.000285


In [158]:
avaliacaoMAP(True,top_k=20)

MAP@ 20: 0.000174


### Precision

In [159]:
avaliacaoPrecision(True,top_k=5)

Precision: 0.03728813559322031


In [160]:
avaliacaoPrecision(True,top_k=10)

Precision: 0.023389830508474548


In [161]:
avaliacaoPrecision(True,top_k=20)

Precision: 0.014406779661016925


### RR

In [162]:
avaliacaoRR(True)

RR: 0.105085


## Config 0

In [163]:
data = base.txtEmenta.fillna("Em branco")

In [164]:
def preprocess(txt):
    txt = str(txt)
    
    stopwords = nltk.corpus.stopwords.words("portuguese")
   
    terms = word_tokenize(txt)
    terms = [word for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [165]:
data_preprocess = Parallel(n_jobs=4)(delayed(preprocess)(doc) for doc in data)

In [166]:
a = time.time()

In [167]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [168]:
d = time.time()

In [169]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 172.484000


## Avaliação

### Recall

In [170]:
avaliacaoRecall(True,top_k=5)

Duração: 13.852995
Recall: 0.2305084745762712


In [171]:
avaliacaoRecall(True,top_k=10)

Duração: 14.968997
Recall: 0.2677966101694915


In [172]:
avaliacaoRecall(True,top_k=20)

Duração: 15.359007
Recall: 0.33220338983050846


### MAP

In [173]:
avaliacaoMAP(True,top_k=5)

MAP@ 5: 0.000553


In [174]:
avaliacaoMAP(True,top_k=10)

MAP@ 10: 0.000308


In [175]:
avaliacaoMAP(True,top_k=20)

MAP@ 20: 0.000288


### Precision

In [176]:
avaliacaoPrecision(True,top_k=5)

Precision: 0.046101694915254184


In [177]:
avaliacaoPrecision(True,top_k=10)

Precision: 0.026779661016949112


In [178]:
avaliacaoPrecision(True,top_k=20)

Precision: 0.01661016949152539


### RR

In [179]:
avaliacaoRR(True)

RR: 0.132203


# TxtIndexacao

In [180]:
data = base.txtIndexacao.fillna("Em branco")

## Config 1

In [181]:
a = time.time()

In [182]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [183]:
d = time.time()

In [184]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 383.970001


## Avaliação

### Recall

In [185]:
avaliacaoRecall(False,top_k=5)

Duração: 23.201962
Recall: 0.17627118644067796


In [186]:
avaliacaoRecall(False,top_k=10)

Duração: 15.532997
Recall: 0.23389830508474577


In [187]:
avaliacaoRecall(False,top_k=20)

Duração: 16.543998
Recall: 0.2745762711864407


### MAP

In [188]:
avaliacaoMAP(False,top_k=5)

MAP@ 5: 0.000588


In [189]:
avaliacaoMAP(False,top_k=10)

MAP@ 10: 0.000495


In [190]:
avaliacaoMAP(False,top_k=20)

MAP@ 20: 0.000292


### Precision

In [191]:
avaliacaoPrecision(False,top_k=5)

Precision: 0.03525423728813558


In [192]:
avaliacaoPrecision(False,top_k=10)

Precision: 0.023389830508474548


In [193]:
avaliacaoPrecision(False,top_k=20)

Precision: 0.013728813559322012


### RR

In [194]:
avaliacaoRR(False)

RR: 0.101695


## Config 5

In [195]:
data = base.txtIndexacao.fillna("Em branco")

In [196]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]

    return " ".join(terms)

In [197]:
data_preprocess = Parallel(n_jobs=4)(delayed(preprocess)(doc) for doc in data)

In [198]:
a = time.time()

In [199]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [200]:
d = time.time()

In [201]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 427.946000


## Avaliação

### Recall

In [202]:
avaliacaoRecall(True,top_k=5)

Duração: 61.922999
Recall: 0.13898305084745763


In [203]:
avaliacaoRecall(True,top_k=10)

Duração: 94.694038
Recall: 0.19661016949152543


In [204]:
avaliacaoRecall(True,top_k=20)

Duração: 95.274000
Recall: 0.24067796610169492


### MAP

In [205]:
avaliacaoMAP(True,top_k=5)

MAP@ 5: 0.000523


In [206]:
avaliacaoMAP(True,top_k=10)

MAP@ 10: 0.000442


In [207]:
avaliacaoMAP(True,top_k=20)

MAP@ 20: 0.000294


### Precision

In [208]:
avaliacaoPrecision(True,top_k=5)

Precision: 0.027796610169491535


In [209]:
avaliacaoPrecision(True,top_k=10)

Precision: 0.019661016949152527


In [210]:
avaliacaoPrecision(True,top_k=20)

Precision: 0.01203389830508473


### RR

In [211]:
avaliacaoRR(True)

RR: 0.067797


## Config 0

In [212]:
data = base.txtIndexacao.fillna("Em branco")

In [213]:
def preprocess(txt):
    txt = str(txt)
    
    stopwords = nltk.corpus.stopwords.words("portuguese")
   
    terms = word_tokenize(txt)
    terms = [word for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [214]:
data_preprocess = Parallel(n_jobs=4)(delayed(preprocess)(doc) for doc in data)

In [215]:
a = time.time()

In [216]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [217]:
d = time.time()

In [218]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 186.232000


## Avaliação

### Recall

In [219]:
avaliacaoRecall(True,top_k=5)

Duração: 14.097000
Recall: 0.2


In [220]:
avaliacaoRecall(True,top_k=10)

Duração: 10.370000
Recall: 0.24067796610169492


In [221]:
avaliacaoRecall(True,top_k=20)

Duração: 125.233998
Recall: 0.3016949152542373


### MAP

In [222]:
avaliacaoMAP(True,top_k=5)

MAP@ 5: 0.000778


In [223]:
avaliacaoMAP(True,top_k=10)

MAP@ 10: 0.000476


In [224]:
avaliacaoMAP(True,top_k=20)

MAP@ 20: 0.000309


### Precision

In [225]:
avaliacaoPrecision(True,top_k=5)

Precision: 0.039999999999999966


In [226]:
avaliacaoPrecision(True,top_k=10)

Precision: 0.02406779661016946


In [227]:
avaliacaoPrecision(True,top_k=20)

Precision: 0.015084745762711838


### RR

In [228]:
avaliacaoRR(True)

RR: 0.101695


# Emenda + Indexacao

In [229]:
base['txtIndexacao'] = base['txtEmenta'] + base['txtIndexacao']

In [230]:
data = base.txtIndexacao.fillna("Em branco")

## Config 1

In [231]:
a = time.time()

In [232]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [233]:
d = time.time()

In [234]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 777.088994


## Avaliação

### Recall

In [235]:
avaliacaoRecall(False,top_k=5)

Duração: 26.311000
Recall: 0.25084745762711863


In [236]:
avaliacaoRecall(False,top_k=10)

Duração: 23.441997
Recall: 0.29152542372881357


In [237]:
avaliacaoRecall(False,top_k=20)

Duração: 28.678998
Recall: 0.34576271186440677


### MAP

In [238]:
avaliacaoMAP(False,top_k=5)

MAP@ 5: 0.000642


In [239]:
avaliacaoMAP(False,top_k=10)

MAP@ 10: 0.000360


In [240]:
avaliacaoMAP(False,top_k=20)

MAP@ 20: 0.000267


### Precision

In [241]:
avaliacaoPrecision(False,top_k=5)

Precision: 0.05016949152542366


In [242]:
avaliacaoPrecision(False,top_k=10)

Precision: 0.029152542372881306


In [243]:
avaliacaoPrecision(False,top_k=20)

Precision: 0.017288135593220306


### RR

In [244]:
avaliacaoRR(False)

RR: 0.162712


## Config 5

In [245]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]

    return " ".join(terms)

In [246]:
data_preprocess = Parallel(n_jobs=4)(delayed(preprocess)(doc) for doc in data)

In [247]:
a = time.time()

In [248]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [249]:
d = time.time()

In [250]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 894.737002


## Avaliação

### Recall

In [251]:
avaliacaoRecall(True,top_k=5)

Duração: 28.082002
Recall: 0.2135593220338983


In [252]:
avaliacaoRecall(True,top_k=10)

Duração: 25.989999
Recall: 0.2677966101694915


In [253]:
avaliacaoRecall(True,top_k=20)

Duração: 29.313998
Recall: 0.3220338983050847


### MAP

In [254]:
avaliacaoMAP(True,top_k=5)

MAP@ 5: 0.000630


In [255]:
avaliacaoMAP(True,top_k=10)

MAP@ 10: 0.000381


In [256]:
avaliacaoMAP(True,top_k=20)

MAP@ 20: 0.000300


### Precision

In [257]:
avaliacaoPrecision(True,top_k=5)

Precision: 0.04271186440677962


In [258]:
avaliacaoPrecision(True,top_k=10)

Precision: 0.026779661016949112


In [259]:
avaliacaoPrecision(True,top_k=20)

Precision: 0.016101694915254205


### RR

In [260]:
avaliacaoRR(True)

RR: 0.118644


## Config 0

In [261]:
def preprocess(txt):
    txt = str(txt)
    
    stopwords = nltk.corpus.stopwords.words("portuguese")
   
    terms = word_tokenize(txt)
    terms = [word for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [262]:
data_preprocess = Parallel(n_jobs=4)(delayed(preprocess)(doc) for doc in data)

In [263]:
a = time.time()

In [264]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [265]:
d = time.time()

In [266]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 899.935999


## Avaliação

### Recall

In [267]:
avaliacaoRecall(True,top_k=5)

Duração: 25.925002
Recall: 0.2576271186440678


In [268]:
avaliacaoRecall(True,top_k=10)

Duração: 29.060002
Recall: 0.30847457627118646


In [269]:
avaliacaoRecall(True,top_k=20)

Duração: 26.565998
Recall: 0.34576271186440677


### MAP

In [270]:
avaliacaoMAP(True,top_k=5)

MAP@ 5: 0.000612


In [271]:
avaliacaoMAP(True,top_k=10)

MAP@ 10: 0.000362


In [272]:
avaliacaoMAP(True,top_k=20)

MAP@ 20: 0.000257


### Precision

In [273]:
avaliacaoPrecision(True,top_k=5)

Precision: 0.05152542372881348


In [274]:
avaliacaoPrecision(True,top_k=10)

Precision: 0.030847457627118588


In [275]:
avaliacaoPrecision(True,top_k=20)

Precision: 0.017288135593220306


### RR

In [276]:
avaliacaoRR(True)

RR: 0.152542
